In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [394]:
import pandas as pd
import numpy as np

hts_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding='euc-kr')
hts_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding='euc-kr')
hts_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding='euc-kr')
hts_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding='euc-kr')
hts = pd.concat([hts_2018,hts_2019,hts_2020,hts_2021], ignore_index=True)

player_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding='euc-kr')
player_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding='euc-kr')
player_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding='euc-kr')
player_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding='euc-kr')
player = pd.concat([player_2018,player_2019,player_2020,player_2021], ignore_index=True)

hitter_base_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2018.csv', encoding='euc-kr')
hitter_base_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2019.csv', encoding='euc-kr')
hitter_base_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2020.csv', encoding='euc-kr')
hitter_base_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2021.csv', encoding='euc-kr')
hitter_base = pd.concat([hitter_base_2018,hitter_base_2019,hitter_base_2020,hitter_base_2021], ignore_index=True)

team = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv', encoding='euc-kr')
schedule_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv', encoding='euc-kr')

In [395]:
# 타자 기록이 있는 선수들과 등록 선수를 pcode와 연도를 기준으로 merge
hitter = hitter_base.merge(player, on=['PCODE','GYEAR'])

In [396]:
# 이름을 개명한 선수에 대해서 같은 이름이 쓰이도록 처리
hitter.loc[hitter['NAME']=='한동민','NAME'] = '한유섬'
hitter.loc[hitter['NAME']=='백민기','NAME'] = '백동훈'
hitter.loc[hitter['NAME']=='윤대영','NAME'] = '윤형준'
hitter.loc[hitter['NAME']=='이병휘','NAME'] = '이유찬'
hitter.loc[hitter['NAME']=='나종덕','NAME'] = '나균안'

In [397]:
###############
# BeautifulSoup

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 데이터를 모을 dataframe 생성
data = pd.DataFrame(columns=['GYEAR','PCODE','NAME','T_ID','POSITION','day', 'vs', 'inn', 'pitcher', 'batter', 'P', 'result', 'before', 'after', 'LEV', 'REa', 'WPs','WPe', 'WPa'])
hitter['BIRTH']=0
same_name_fail_list = []

for hitter_index in tqdm(range(len(hitter))):
  year = hitter.loc[hitter_index,'GYEAR']
  player_pcode = hitter.loc[hitter_index,'PCODE']
  player_name = hitter.loc[hitter_index,'NAME']
  player_team = hitter.loc[hitter_index,'T_ID']
  player_position = hitter.loc[hitter_index,'POSITION']

  # 선수와 연도에 맞는 플레이 로그 페이지
  wp = requests.get('http://www.statiz.co.kr/player.php?opt=6&sopt=0&name=' + player_name + '&da=1&year=' + str(year))
  soup = BeautifulSoup(wp.text, "html.parser")

  # 동명이인 문제로 페이지에 데이터가 없다면
  row_data = soup.find_all(class_='table table-striped table-responsive table-condensed table-bordered')
  if len(row_data)==0:
    print("\n동명이인 존재 : " + player_name)
    if hitter.loc[hitter_index,'BIRTH']!=0:
      print("생일로 찾음")
      wp = requests.get('http://www.statiz.co.kr/player.php?opt=6&sopt=0&name=' + player_name + '&birth=' + hitter.loc[hitter_index,'BIRTH'] + '&da=1&year=' + str(year))
      soup = BeautifulSoup(wp.text, "html.parser")
    else:  
      same_name_href_list=[]
      wp = requests.get('http://www.statiz.co.kr/player.php?name=' + player_name)
      soup = BeautifulSoup(wp.text, "html.parser")
      for same_name_list in soup.find(class_='table table-striped').find_all('a'):
        same_name_href_list.append(same_name_list.attrs['href'])
      flag=0
      for same_name_page in same_name_href_list:
        wp = requests.get('http://www.statiz.co.kr/'+same_name_page+'&opt=1')
        soup = BeautifulSoup(wp.text, "html.parser")
        # 현재 찾고 있는 선수의 기록과 현재 페이지의 선수 기록이 일치하면 같은 선수라는 의미이다. 
        d = str(hitter.loc[hitter_index,'GAMENUM'].item()) + str(hitter.loc[hitter_index,'PA'].item()) + str(hitter.loc[hitter_index,'AB'].item())
        if d in soup.text:
          flag=1
          # 동명이인의 선수에 대해서 생일을 기록해둔다.
          print(player_name+" 생일 저장")
          player_birth = same_name_page[-10:]
          hitter.loc[hitter['PCODE']==hitter.loc[hitter_index,'PCODE'], 'BIRTH'] = player_birth
          break
      # 찾은 선수의 페이지로 접근한다.
      if flag==1:
        print("기록과 일치하는 선수 찾음")
      else:
        print("기록과 일치하는 선수 못찾음")
        same_name_fail_list.append((year,player_pcode,player_name,player_team))
        continue
      wp = requests.get('http://www.statiz.co.kr/player.php?opt=6&'+same_name_page.split('?')[1]+'&da=1&year='+str(year))
      soup = BeautifulSoup(wp.text, "html.parser")

  # 데이터 테이블 접근
  row_data = soup.find_all(class_='table table-striped table-responsive table-condensed table-bordered')[0]
  row_data_tr = row_data.find_all('tr')[1:]

  # 행렬만들기
  rawdata_day = np.zeros(len(row_data_tr), dtype=object)
  rawdata_vs = np.zeros(len(row_data_tr), dtype=object)
  rawdata_inn = np.zeros(len(row_data_tr), dtype=object)
  rawdata_pitcher = np.zeros(len(row_data_tr),dtype=object)
  rawdata_batter = np.zeros(len(row_data_tr), dtype=object)
  rawdata_P = np.zeros(len(row_data_tr), dtype=object)
  rawdata_result = np.zeros(len(row_data_tr), dtype=object)
  rawdata_before = np.zeros(len(row_data_tr), dtype=object)
  rawdata_after = np.zeros(len(row_data_tr), dtype=object)
  rawdata_LEV = np.zeros(len(row_data_tr), dtype=object)
  rawdata_REa = np.zeros(len(row_data_tr), dtype=object)
  rawdata_WPs = np.zeros(len(row_data_tr), dtype=object)
  rawdata_WPe = np.zeros(len(row_data_tr), dtype=object)
  rawdata_WPa = np.zeros(len(row_data_tr), dtype=object)

  # 행 하나씩 접근, 헤더 생략
  for tr_index in range(len(row_data_tr)):
    rawdata_td_element = [text for text in row_data_tr[tr_index].stripped_strings]
    # P 칼럼의 값이 없을때도 있음.
    if len(rawdata_td_element) == 14:
      rawdata_td_element.insert(6,"")
    
    # 데이터 입력
    rawdata_day[tr_index] = rawdata_td_element[0]
    rawdata_vs[tr_index] = rawdata_td_element[1]
    rawdata_inn[tr_index] = rawdata_td_element[2]
    rawdata_pitcher[tr_index] = rawdata_td_element[3]
    rawdata_batter[tr_index] = rawdata_td_element[5]
    rawdata_P[tr_index] = rawdata_td_element[6]
    rawdata_result[tr_index] = rawdata_td_element[7]
    rawdata_before[tr_index] = rawdata_td_element[8]
    rawdata_after[tr_index] = rawdata_td_element[9]
    rawdata_LEV[tr_index] = rawdata_td_element[10]
    rawdata_REa[tr_index] = rawdata_td_element[11]
    rawdata_WPs[tr_index] = rawdata_td_element[12]
    rawdata_WPe[tr_index] = rawdata_td_element[13]
    rawdata_WPa[tr_index] = rawdata_td_element[14]

  tmp_data = pd.DataFrame()

  # tmp_data는 각 선수마다 데이터를 모은 후 data에 concat 하기 전 임시로 저장
  tmp_data['day'] = rawdata_day
  tmp_data['vs'] = rawdata_vs
  tmp_data['inn'] = rawdata_inn
  tmp_data['pitcher'] = rawdata_pitcher
  tmp_data['batter'] = rawdata_batter
  tmp_data['P'] = rawdata_P
  tmp_data['result'] = rawdata_result
  tmp_data['before'] = rawdata_before
  tmp_data['after'] = rawdata_after
  tmp_data['LEV'] = rawdata_LEV
  tmp_data['REa'] = rawdata_REa
  tmp_data['WPs'] = rawdata_WPs
  tmp_data['WPe'] = rawdata_WPe
  tmp_data['WPa'] = rawdata_WPa

  tmp_data['GYEAR'] = year
  tmp_data['PCODE'] = player_pcode
  tmp_data['NAME'] = player_name
  tmp_data['T_ID'] = player_team
  tmp_data['POSITION'] = player_position

  # 데이터 쌓기
  data = pd.concat([data,tmp_data], ignore_index=True)

  0%|          | 3/1098 [00:02<15:12,  1.20it/s]


동명이인 존재 : 이해창
이해창 생일 저장
기록과 일치하는 선수 찾음


  1%|          | 7/1098 [00:10<29:43,  1.63s/it]


동명이인 존재 : 이상호
이상호 생일 저장
기록과 일치하는 선수 찾음


  1%|          | 8/1098 [00:15<44:37,  2.46s/it]

...

 99%|█████████▉| 1090/1098 [34:57<00:37,  4.75s/it]


동명이인 존재 : 박건우
생일로 찾음


100%|█████████▉| 1095/1098 [35:14<00:09,  3.32s/it]


동명이인 존재 : 김상수
생일로 찾음


100%|██████████| 1098/1098 [35:26<00:00,  1.94s/it]


In [398]:
data

,GYEAR,PCODE,NAME,T_ID,POSITION,day,vs,inn,pitcher,batter,P,result,before,after,LEV,REa,WPs,WPe,WPa
0,2018,60100,백창수,HH,외,2018-03-27,NC,8초,이민호,백창수,1-0,유격수 땅볼,무사 6:9,1사 6:9,0.88,-0.258,93.1,95.3,-0.022
1,2018,60100,백창수,HH,외,2018-03-28,NC,2초,구창모,백창수,2-3,볼넷,2사 2루 0:2,"2사 1,2루 0:2",1.10,0.122,71.9,70.7,0.012
2,2018,60100,백창수,HH,외,2018-03-28,NC,4초,구창모,백창수,0-0,3루수 땅볼,2사 1:2,이닝종료 1:2,0.51,-0.117,67.4,68.8,-0.013
3,2018,60100,백창수,HH,외,2018-03-28,NC,7초,유원상,백창수,2-3,유격수 땅볼 실책,[실책포함],무사 2:2,무사 1루 2:2,1.54,0.398,50.0,44.1
4,2018,60100,백창수,HH,외,2018-03-30,SK,2말,김태훈,백창수,0-1,백창수/10473 : 2루수 직선타 아웃,"1사 2,3루 1:1","2사 2,3루 1:1",1.57,-0.833,64.1,56.1,-0.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207784,2021,79608,안치홍,LT,내,2021-08-22,KT,7말,고영표,안치홍,2-3,2루수 땅볼,무사 3:0,1사 3:0,1.07,-0.258,12.4,9.6,-0.027
207785,2021,79608,안치홍,LT,내,2021-08-22,KT,9말,김재윤,안치홍,2-3,볼넷,1사 3:0,1사 1루 3:0,0.50,0.276,1.8,4.5,0.027
207786,2021,79608,안치홍,LT,내,2021-08-23,KT,1말,데스파이네,안치홍,0-2,좌익수 안타,"1사 1,2루 0:1","1사 1,2루 0:2",1.64,1.000,67.9,76.1,0.082
207787,2021,79608,안치홍,LT,내,2021-08-23,KT,3말,데스파이네,안치홍,1-1,1루수 땅볼,무사 0:4,1사 0:4,0.30,-0.258,89.7,89.0,-0.008


In [401]:
data.to_csv('/content/drive/MyDrive/crawling_data')